In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

### Imports

In [ ]:
import os
import gc
import cv2
import sys
import tifffile
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from collections import Counter
from matplotlib import pyplot as plt

sys.path.append("../code/")

In [ ]:
from data.dataset import load_image
from utils.rle import *
from params import *

### Preparation

In [ ]:
FACTOR = 4

In [ ]:
out_dir = DATA_PATH + f"train_{FACTOR}/"

In [ ]:
df_masks = pd.read_csv(DATA_PATH + "train.csv").set_index("id")

In [ ]:
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

### Loop

In [ ]:
masks = {}

for index, encs in tqdm(df_masks.iterrows(), total=len(df_masks)):
    # read image and generate the mask
    img = load_image(os.path.join(TIFF_PATH, index + ".tiff"))
    mask = enc2mask(encs, (img.shape[1], img.shape[0]))

    img = cv2.resize(
        img,
        (img.shape[1] // FACTOR, img.shape[0] // FACTOR),
        interpolation=cv2.INTER_AREA,
    )

    mask = cv2.resize(
        mask,
        (mask.shape[1] // FACTOR, mask.shape[0] // FACTOR),
        interpolation=cv2.INTER_NEAREST,
    )
    
    rle = mask2enc(mask)
    
    masks[index] = rle
    
    tifffile.imsave(out_dir + f"{index}.tiff", img)

In [ ]:
df_masks = pd.DataFrame.from_dict(masks).T.reset_index().rename(columns={0: "encoding", "index": "id"})

In [11]:
from params import DATA_PATH
df_masks.to_csv(f"{DATA_PATH}train_{FACTOR}.csv", index=False)

## LAB Stats

In [9]:
from data.transforms import get_lab_stats

In [10]:
df_masks = pd.read_csv(f"../input/train_{FACTOR}.csv")

In [12]:
stats_dic = {}

for index, encs in tqdm(df_masks.values):
    # read image and generate the mask
    img = load_image(os.path.join(out_dir, index + ".tiff"), full_size=False)
    
    stats = get_lab_stats(img)
    
    stats_dic[index] = stats
    
    del img
    gc.collect()

# SAVED STATS


{'2f6ecfcdf': (array([145.20238231, 137.9992975 , 118.50750566]),
  array([89.18389495, 14.00093526, 13.21949608])),
 '8242609fa': (array([160.79001264, 142.6352168 , 114.00284024]),
  array([70.24989634, 15.22372627, 14.4514529 ])),
 'aaa6a05cc': (array([160.45423395, 143.03174444, 114.09910137]),
  array([74.09723191, 17.26439303, 15.81758684])),
 'cb2d976f4': (array([149.31195412, 140.31331355, 117.20462794]),
  array([84.94398285, 15.75064753, 13.52656749])),
 'b9a3865fc': (array([171.01812023, 143.30085418, 113.43859791]),
  array([59.4382794 , 14.68882715, 13.63255176])),
 'b2dc8411c': (array([140.90416801, 138.17195697, 118.23776168]),
  array([89.70731926, 13.98473528, 13.16651675])),
 '0486052bb': (array([150.98582581, 137.75751632, 118.39993127]),
  array([86.3467948 , 13.35411092, 12.92842957])),
 'e79de561c': (array([157.95985203, 146.4938866 , 111.56713176]),
  array([54.46774812, 15.66899484, 14.39054206])),
 '095bf7a1f': (array([129.59920559, 144.70288534, 116.08658614]),
  array([79.32734831, 18.45795296, 13.04625025])),
 '54f2eec69': (array([149.58838888, 143.2456009 , 116.12248584]),
  array([73.62551546, 17.1735228 , 13.98801728])),
 '4ef6695ce': (array([152.16802057, 148.97719682, 112.45433171]),
  array([63.19192052, 19.56262307, 14.6277392 ])),
 '26dc41664': (array([135.29452738, 146.14188243, 114.37756163]),
  array([77.62458547, 19.65603636, 14.95319517])),
 'c68fe75ea': (array([164.85067316, 147.15755839, 112.1557135 ]),
  array([42.72894554, 14.53008223, 12.74913869])),
 'afa5e8098': (array([135.90040118, 143.43050015, 114.02753278]),
  array([70.04322744, 13.57595383, 12.60366562])),
 '1e2425f28': (array([133.55795968, 153.76124745, 111.09582196]),
  array([66.04495908, 20.47925883, 13.85287878]))}